In [42]:
# basics
import intake
import xarray as xr
import dask  # memory-efficient parallel computation and delayed execution (lazy evaluation).
import subprocess as sp

%run gem_helpers.ipynb

uid = getpass.getuser()
image_path = make_tempdir("intake_demo_plots")
data_cache_path = make_tempdir("intake_demo_data")

# this function is also part of the gem helpers
def get_list_from_cat(catalog, column):
    """A helper function for getting the contents of a column in an intake catalog.
    Call with the catalog to be inspected and the column of interest."""
    return sorted(catalog.unique(column)[column]["values"])

catalog_file = "/work/ka1081/Catalogs/dyamond-nextgems.json"
cat = intake.open_esm_datastore(catalog_file)


In [58]:
##check and find exp and var
fre  = "3hour" #30minute
var  = "hfls" #tas, sst
rea  = "atm"
hits = cat.search(simulation_id=["ngc2009", "ngc2012","ngc2013"],  realm=rea, variable_id= var, frequency=fre)
get_from_cat(hits,["grid_label","grid_id","simulation_id", "variable_id",'frequency'])

,grid_label,grid_id,simulation_id,variable_id,frequency
0,gn,5aff0578-9bd9-11e8-8e4a-af3d880818e6,ngc2013,"(sfcwind, clivi, cllvi, cptgzvi, hfls, hfss, prlr, pr, prw, qgvi, qrvi, qsvi, rlds, rlus, rlut, rsds, rsdt, rsus, rsut, tauu, tauv, rpds_dir, rpds_dif, rvds_dif, rnds_dif, evspsbl)",3hour
1,gn,not implemented,ngc2012,"(sfcwind, clivi, cllvi, cptgzvi, hfls, hfss, prlr, pr, prw, qgvi, qrvi, qsvi, rlds, rlus, rlut, rsds, rsdt, rsus, rsut, tauu, tauv, rpds_dir, rpds_dif, rvds_dif, rnds_dif, evspsbl)",3hour


In [63]:
simulation_id=["ngc2012","ngc2013"]
sim= simulation_id[1]
fre= "3hour"
var= "hfls"
catalog=hits.search(simulation_id=sim,frequency=fre, variable_id=var)
files=sorted(catalog.unique("uri")["uri"]["values"])

outfile = f"{data_cache_path}/{var}_daymean_{sim}.nc"

In [64]:
outfile

'/scratch/b/b381639/intake_demo_data/hfls_daymean_ngc2013.nc'

In [65]:



    if True:
        query = (
            [
                "cdo",
                "-P",
                "4",
#                "-remapcon,r360x180",
                "-daymean",
                "-sellonlatbox,300,310,5,17",   #-sellonlatbox,0,360,-30,30"
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + files #[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
#    outfile_dict[simulation_id] = outfile

['cdo', '-P', '4', '-daymean', '-sellonlatbox,300,310,5,17', '-cat', '-apply,-selname,hfls', '[', '/work/bm1235/k203123/experiments/ngc2013/run_20200120T000000-20200131T235830/ngc2013_atm_2d_3h_mean_20200120T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2013/run_20200120T000000-20200131T235830/ngc2013_atm_2d_3h_mean_20200201T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2d_3h_mean_20200201T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2d_3h_mean_20200301T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2d_3h_mean_20200401T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2d_3h_mean_20200501T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2013/run_20200201T000000-20200531T235830/ngc2013_atm_2d_3h_mean_20200601T000000Z.nc', '/work/bm1235/k203123/experiments/ngc2013

In [24]:
for files in file_cat:
    fileS=files.split(',')

In [25]:
print(fileS)

['/work/bm1235/k203123/experiments/ngc2012/run_20271101T000000-20280229T235830/ngc2012_atm_2d_3h_mean_20280201T000000Z.nc']


In [26]:
for files in file_cat:
    fileS=files.split(',')
    outfile = f"{data_cache_path}/{var}_daymean_{simulation_id}.nc"
    if True:
        query = (
            [
                "cdo",
                "-P",
                "4",
                "-remapcon,r360x180",
                "-daymean",
                "-sellonlatbox,300,310,5,17",   #-sellonlatbox,0,360,-30,30"
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + fileS[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
    outfile_dict[simulation_id] = outfile

['cdo', '-P', '4', '-remapcon,r360x180', '-daymean', '-sellonlatbox,300,310,5,17', '-cat', '-apply,-selname,sfcwind', '[', '/work/bm1235/k203123/experiments/ngc2012/run_20200120T000000-20200131T235830/ngc2012_atm_2d_3h_mean_20200120T000000Z.nc', ']', '/scratch/b/b381639/intake_demo_data/sfcwind_daymean_ngc2012.nc']
cdo(1) daymean: Process started
cdo(2) sellonlatbox: Process started
cdo(3) cat: Process started
cdo(4) selname: Process started
cdo    remapcon: YAC first order conservative weights from unstructured (14701) to lonlat (360x180) grid
cdo(4) selname: Processed 503316480 values from 26 variables over 96 timesteps.
cdo(3) cat: Processed 503316480 values from 1 variable over 96 timesteps.
cdo(2) sellonlatbox: Processed 503316480 values from 1 variable over 96 timesteps.
cdo(1) daymean: Processed 1411296 values from 1 variable over 107 timesteps.
cdo    remapcon: Processed 176412 values from 1 variable over 12 timesteps [10.88s 2043MB].


NameError: name 'outfile_dict' is not defined

In [16]:
        query = (
            [
                "cdo",
                "-P",
                "4",
                "-remapcon,r360x180",
                "-daymean",
                "-sellonlatbox,300,310,5,17",   #-sellonlatbox,0,360,-30,30"
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + files[:10]
            + ["]", outfile]
        )

NameError: name 'files' is not defined

In [71]:
outfile_dict = {}
for simulation_id, files in file_cat.items():
    outfile = f"{data_cache_path}/{var}_daymean_{simulation_id}.nc"
    if True: # not os.access(outfile, os.R_OK):
        query = (
            [
                "cdo",
                "-P",
                "4",
                "-remapcon,r360x180",
                "-daymean",
                "-sellonlatbox,300,310,5,17",   #-sellonlatbox,0,360,-30,30"
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + files[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
    outfile_dict[simulation_id] = outfile

'/scratch/b/b381639/intake_demo_data/sfcwind_daymean_1.nc'

In [19]:
file_cat = {}
#for simulation_id in ("ngc2009", "ngc2012"):
for simulation_id in ("ngc2009","ngc2012"):
    file_cat[simulation_id] = get_list_from_cat(
        hits.search(simulation_id=simulation_id), "uri"
    )

outfile_dict = {}
for simulation_id, files in file_cat.items():
    outfile = f"{data_cache_path}/{var}_daymean_{simulation_id}.nc"
    if True: # not os.access(outfile, os.R_OK):
        query = (
            [
                "cdo",
                "-P",
                "4",
                "-remapcon,r360x180",
                "-daymean",
                "-sellonlatbox,300,310,5,17",   #-sellonlatbox,0,360,-30,30"
                "-cat",
                f"-apply,-selname,{var}",
                "[",
            ]
            + files[:10]
            + ["]", outfile]
        )
        # Note, we only use the first 10 files to save time (the [:10] in files[:10]). Remove the [:10] to compute over the whole experiment.
        print(query)
        sp.run(query)
    outfile_dict[simulation_id] = outfile

/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/intake_esm/search.py:106: UserWarning: Query returned zero results.
  warn(message)
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/intake_esm/search.py:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  x = df[columns].apply(_find_unique, result_type='reduce').to_dict()


ValueError: Length of values (0) does not match length of index (1)